# SEGMENTING AND CLUSTERING NEIGHBOURHOODS IN TORONTO 

## PRE PROCESSING AND CONVERTING DATA INTO DATA FRAME 

In [1]:
from bs4 import BeautifulSoup
import requests   # library to handle requests
import lxml       # parse the website in lxml format
import numpy as np
import pandas as pd

In [6]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_='wikitable sortable')
# print(table.prettify())

In [7]:
# headers="Postcode,Borough,Neighbourhood"
table1 = ""
for tr in table.find_all('tr'):
    row = ""
    for tds in tr.find_all('td'):
        row = row + "," + tds.text
    table1 = table1 + row[1:]
# print(table1)

In [8]:
csv_file = open('toronto.csv', 'wb')
csv_file.write(bytes(table1,encoding="ascii",errors="ignore"))

6666

In [9]:
col_names = ['Postalcode', 'Borough', 'Neighbourhood']
df = pd.read_csv('toronto.csv' , names=col_names)

df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,NaN,Not assigned,NaN
2,NaN,NaN,NaN
3,M2A,NaN,NaN
4,NaN,Not assigned,NaN


In [10]:
indexNum = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNum, inplace = True)
df.head(10)

,Postalcode,Borough,Neighbourhood
0,M1A,NaN,NaN
2,NaN,NaN,NaN
3,M2A,NaN,NaN
5,NaN,NaN,NaN
6,M3A,NaN,NaN
7,NaN,North York,NaN
8,NaN,Parkwoods,NaN
9,M4A,NaN,NaN
10,NaN,North York,NaN
11,NaN,Victoria Village,NaN


In [11]:
# If a cell has a borough but a Not assigned neighborhood
# then the neighborhood will be the same as the borough

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,NaN,NaN
2,NaN,NaN,NaN
3,M2A,NaN,NaN
5,NaN,NaN,NaN
6,M3A,NaN,NaN


In [12]:
df.shape

(463, 3)

## CREATE DATAFRAME WITH LONGITUDE AND LATITUDE VALUES 

In [13]:
# using a csv file that has the geographical coordinates
!wget -q -O 'Toronto_location.csv'  http://cocl.us/Geospatial_data
df_loc = pd.read_csv('Toronto_location.csv')
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_loc.columns = ['Postalcode','Latitude','Longitude']
df_loc.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_toronto = pd.merge(df, df_loc, on = 'Postalcode')
df_toronto.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,NaN,NaN,43.753259,-79.329656
1,M4A,NaN,NaN,43.725882,-79.315572
2,M5A,NaN,NaN,43.654260,-79.360636
3,M6A,NaN,NaN,43.718518,-79.464763
4,M7A,NaN,NaN,43.662301,-79.389494


## EXPLORING AND CLUSTERING NEIGHBOURHOOD

In [2]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [3]:
# Geograpical coordinate of Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:

map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [35]:
CLIENT_ID = 'N3JZIOYSXFEDAYISCE3ZDMGEYXNBPIQNFUHPWPVEZPOWEQ3E' # your Foursquare ID
CLIENT_SECRET = 'NKO0ERQU2LFFKWZYFEXKDTHOQIWRZBIFBKUEDXDJODGD442U' # your Foursquare Secret
VERSION = '20191210' # Foursquare API version

# defining radius and limit of venues to get
radius=500
LIMIT=100

In [78]:
 def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name) # to check if the algorithm is working on all elements
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], 
            v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 'Latitude', 'Longitude', 
                  'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [87]:
toronto_venues = getNearbyVenues(names = df_toronto['Postalcode'], latitudes = df_toronto['Latitude'], longitudes = df_toronto['Longitude'])
toronto_venues.head()

,Postcode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [39]:
toronto_venues.shape

(2127, 7)

In [89]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Postalcode'] = toronto_venues['Postcode'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
#toronto_onehot.head()

(2126, 276)


In [42]:
toronto_onehot.shape

(2127, 275)

In [104]:
toronto_grouped = toronto_onehot.groupby('Postalcode').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(98, 276)


,Postalcode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num = 10
indicators = ['st', 'nd', 'rd']
columns = ['Postalcode']
for ind in np.arange(num):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postalcode'] = toronto_grouped['Postalcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = most_common_venues(toronto_grouped.iloc[ind, :], num)

neighborhoods_venues_sorted.head()

,Postalcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Print Shop,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
1,M1C,Bar,History Museum,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
2,M1E,Electronics Store,Mexican Restaurant,Rental Car Location,Intersection,Breakfast Spot,Medical Center,Bank,Diner,Discount Store,Distribution Center
3,M1G,Coffee Shop,Indian Restaurant,Korean Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega
4,M1H,Hakka Restaurant,Gas Station,Bank,Bakery,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Lounge,Coworking Space


In [124]:
kclusters = 5 # number of clusters

toronto_grouped_clustering = toronto_grouped.drop('Postalcode', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:5] #check the first 10 cluster labels generated



toronto_merged.head()


,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,NaN,NaN,43.753259,-79.329656
1,M4A,NaN,NaN,43.725882,-79.315572
2,M5A,NaN,NaN,43.654260,-79.360636
3,M6A,NaN,NaN,43.718518,-79.464763
4,M7A,NaN,NaN,43.662301,-79.389494


In [132]:
    latitude = 43.425972
longitude = -79.202647
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postalcode']):
     label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        
        fill=True,
        
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters